# <span style="color:blue">Bitcoin and Cryptocurrency Technologies</span>
Platform: Coursera

Institutions: Princeton University

Instructors: Arvind Narayanan

### <span style="color:red">Week 1</span>
##### <span style="color:green">Cryptographic Hash Functions</span>
Hash function:
- takes any string as input
- fixed-size output
- efficiently computable

properties:
- collision-free
- hiding
- puzzle-friendly

Collision-free:

Nobody can find $x,y$ such that $x\neq y$ and $H(x)=H(y)$.

Collisions do exist. No function has been proven to be collision-free.

So if we know $H(x)=H(y)$, it's safe to assume $x=y$. To recognize a file that we saw before, just remember its hash.

Hiding:

Given $H(x)$, it's infeasible to find $x$.

If $r$ is chosen from a probability distribution that has high min-entropy, then given $H(r|x)$, it is infeasible to find $x$. High min-entropy means that the distribution is very spread out, so that no particular value is chosen with more than negligible probability.

Puzzle-friendly:

For every possible output $y$, if $k$ is chosen from a distribution with high min-entropy, then it is infeasbile to find $x$ such that $H(k|x)=y$.

Given $id$ from a high min-entropy distrubution and a target set $Y$, try to find an $x$ such that $H(id|x)\in Y$. Puzzle-friendly property implies that no solving strategy is much better than trying random values of $x$.

SHA-256 hash function
##### <span style="color:green">Hash Pointers and Data Structures</span>
Hash pointer:
- pointer to where some info is stored
- (cryptographic) hash of the info

With a hash pointer, we can
- ask to get the info back
- verify that it hasn't changed

Linked list with hash pointers: Blockchain

Binary tree with hash pointers: Merkle Tree

$O(\log n)$ to prove a membership in a Merkle tree

$O(\log n)$ to prove a non-membership in a Merkle tree

Hash pointers can be used in any pointer-based data structure that has no cycles.
##### <span style="color:green">Digital Signatures</span>
Properties of signatures:
- Only you can sign, but anyone can verify
- tied to a document, can't be cut-and-pasted to another document.

Digital signature APIs:

secret_key, public_key := generateKeys(size)

signature := sign(secret_key, message)

isValid := verify(public_key, message, signature)

Adversary who knows the public key gets to see signatures on messages of his choice can't produce a verifiable signature on another message.

Bitcoin uses ECDSA (Elliptic Curve Digital Signature) standard
##### <span style="color:green">Goofycoin</span>
Goofy can create new coins. (Signed by public_key)

A coin's owner can spend it. (A Hash pointer pointing to the original transaction and signed by the owner)

The recipient can pass it on to someone else.
##### <span style="color:green">Scroogecoin</span>
Scrooge publishes a history of all transactions (a block chain, signed by Scrooge)

CreateCoins transaction (with an ID) creates new coins.

Each coin has a coin id, value, and recipient.

PayCoins transaction consumes (destroys) some coins, and creates new coins of the same total value.

A PayCoin transaction is valid if
- consumed coins are valid
- coins have not already been consumed
- total value out equals to total value in
- signed by owners of all consumed coins

Scrooge coins are immutable and can't be transferred, subdivided, or combined.

### <span style="color:red">Week 2</span>
##### <span style="color:green">Centralization V.S. Decentralization</span>
Scroogecoin relies on centralization (Scrooge).

Decentralization is not all-or-nothing (E-mail: decentralized protocol, centralized web services).

Aspects of decentralization in Bitcoin
- Peer-to-Peer network: open to anyone, low barrier to entry.
- Mining: open to anyone, but inevitable concentration of power.
- Updates to software: core developers trusted by community, have great power.

##### <span style="color:green">Distributed Consensus</span>
Distributed consensus:
- The protocol terminates and all correct nodes decide on the same value.
- The value must have been proposed by some correct node.

Bitcoin is a peer-to-peer system.

A payment broadcasts the transaction to all Bitcoin nodes.

- All nodes have a sequence of blocks of transactions they've reached consensus on.
- Each node has a set of outstanding transactions it's heard about.

Bitcoin introduces incentives and embraces randomness.
##### <span style="color:green">Consensus without Identity: The Blockchain</span>
Nodes have no long-term identities.
- Identity is hard in a P2P system.
- Pseudonymity is a goal of Bitcoin.

Implicit consensus
1. In each round, a random node is picked.
2. This node proposes the next block in the chain.
3. Other nodes implicitly accept (extend it) / reject (ignore it and extend chain from earlier blocks) the block.
4. Every block contains hash of the block it extends.

Consensus algorithm:
1. New transactions are broadcast to all nodes.
2. Each node collects new transactions into a block.
3. In each round a random node gets to broadcast its block.
4. Other nodes accept the block only if all transactions in it are valid (no double-spending, valid signatures).
5. Nodes express their acceptance of the block by including its hash in the next block they create.

Honest nodes will extend the longest valid branch.

Double-spend probability decreases exponentially with the number of confirmations.

Most common heuristic in the Bitcoin community: 6 confirmations

Protection against double-spending is purely by consensus. You're never 100% sure a transaction is in consensus branch. Guarantee is probablistic.
##### <span style="color:green">Incentives and proof of work</span>
We can't penalize faulty nodes as they don't have identities. We can reward honest nodes.

Incentive 1: Block reward

Creator of block gets to
- include special coin-creation transaction in the block.
- choose recipient address of the transaction.

Block creator gets to collect the reward only if the node ends up on long-term consensus branch.

There's a finite supply of bitcoins. (Total supply 21 million. Runs out in 2140.)

Incentive 2: Transaction fee

Creator of transaction can choose to make output value less than input value. Remainder is a transaction fee and goes to block creator.

Proof of work: Select nodes in proportion to a resource that no one can monopolize.

Equivalent views of proof of work
- Select nodes in proportion to computing power.
- Nodes compete for right to create block.
- Moderately hard to create new identities.

Proof of work in Bitcoin: Hash puzzles

To create block, find nonce such that $H(nonce||prev\_hash||tx||...||tx)$ is very small

Properties of proof of work:
1. difficult to compute (Only some nodes bother to compete - miners)
2. parameterizable cost (Nodes automatically re-calculate the target every two week).
3. trivial to verify (Nonce must be published as part of block).

Key security assumption: Attacks infeasible if majority of miners weighted by hash power follow the protocol.

### <span style="color:red">Week 3</span>
##### <span style="color:green">Bitcoin transactions</span>
An account-based ledger might need to scan backwards until genesis.

Bitcoin uses a transaction-based ledger.

Bitcoin always consumes all the coins of the previous transaction. (Finite backward scan for validity)

Joint payment: Combine coins from previous transactions and output the same combined value.

A Bitcoin transaction contains three parts
- metadata
   Hash of entire transaction, size of transaction, lock time, ...
- inputs
   Hash of previous transactions, index of output, signature
- outputs
   Value
   Recipient (script)

##### <span style="color:green">Bitcoin scripts</span>
Output "addresses" are really scripts. Input "addresses" are also scripts.

Each Bitcoin script can only provide two outcomes, either execute successfully or any error.

Proof of burn can never be retrieved.
- can write extra data into Blockchain

Pay to script hash: Senders can simply provide the hash of the script to redeem the coins.
##### <span style="color:green">Bitcoin blocks</span>
Why group transaction into blocks?
- Single unit of work for miners
- Limit length of hash-chain of blocks

Bitcoin block contains hash chain of blocks on the top, and Hash (Merkle tree) of transactions in each block.
##### <span style="color:green">The Bitcoin network</span>
Bitcoin P2P network
- Ad-hoc protocol (runs on TCP port 8333)
- Ad-hoc network with random topology
- All nodes are equal
- New nodes can join at any time
- Forget non-responding nodes after 3 hours

Relay a proposed transaction
- valid with current block chain
- script matches a whitelist
- Haven't seen before
- Doesn't conflict with others relayed.

Race conditions
- Default behavior: accept what you hear first.
- Network position matters.
- Miners may implement other logic.

Size of the Bitcoin network:
- impossible to measure exactly
- estimates-up to 1 million IP addresses / month
- only about 5 - 10k (dropping) "full nodes" (permanently connected and fully validate)

Unspent Transaction Output (UTXO)
##### <span style="color:green">Limits and Improvements</span>
Throughput limits in Bitcoin

- 1 million bytes / block
- &gt;250 bytes / transaction
- 7 transactions / sec (compare to VISA: 2000-10000 transactions / sec, PayPal: 50-100 transactions / sec)

Cryptographic limits in Bitcoin

- 1 signature algorithm (ECDSA/P256)
- Hard-coded hash functions

Crypto primitives may break in 2040.

### <span style="color:red">Week 4</span>
##### <span style="color:green">Store and use Bitcoins</span>
Goals: You can spend your coins. Nobody else can spend your coins. Convinient to spend 

Store key locally in a file:
- Very convinient
- As available and secure as the device

Wallet software: Keep track of coins

Nice trick: Use a separate address / key for each coin
##### <span style="color:green">Hot and Cold Storage</span>
Hot storage (online, convinient but risky)

Cold storage (offline, archival but safer)

Use separate keys for hot and cold storage.

Hot storage has hot secret keys and cold addresses.

Hierarchical key generation generates an "address gen info" and a "private key gen info".

With "address gen info" ("private key gen info") and an integer i, the ith address (key) can be generated.

ECDSA (used by Bitcoin) supports hierarchical key generation.

"Address gen info" can be put in the hot storage, while "private key gen info" is kept in the cold storage.
##### <span style="color:green">Splitting and Sharing Keys</span>
Idea of secret sharing: Split secret into $N$ pieces, such that given any $K$ pieces, can reconstruct the secret given fewer than $K$ pieces, don't learn anything.

Example: 

$N=2$, $K=2$, $P$ is a large number, $S$ is secret in $[0,P)$, $R$ is a random number in $[0,P)$

split: $X_1=(S+R)\mod P$, $X_2=(S+2R)\mod P$

reconstruct: $(2X_1-X_2)\mod P = S$

Secret can be done with such polynomial equations.

Secret sharing:
- Good: Store shares separately, adversary must compromise several shares to get the key.
- Bad: To sign, need to bring shares together.

##### <span style="color:green">Online Wallets and Exchanges</span>
Online wallet is like a local wallet but "in the cloud".

Tradeoff:
- convinient: nothing to install, works on multiple devices
- security worries: vulnerable if site is malicious or compromised

Bitcoin Exchanges accept deposits of Bitcoins and fiat currency.

Proof of reserve: Bitcoin exchange can prove it has fractional reserve by publishing valid payment to self and sign a challenge string with the same private key.

Use Merkle Tree to prove that the account is in the group.
##### <span style="color:green">Payment Services</span>
Typical flow of payment service:
1. The merchant provides the payment button with transID and amount to the user.
2. The user clicks the button and sends transID and amount to the payment service.
3. The payment service sends interaction to user and triggers the browser to send a message back to the merchant.
4. The payment service sends confirmation with transID and amount to the merchant.

##### <span style="color:green">Currency Exchange Markets</span>
Supply of Bitcoins = coins in circulation (+ demand deposits)

coins in circulation: fixed number

When to include demand deposits? When they can actually be sold in the market.

### <span style="color:red">Week 5</span>
##### <span style="color:green">The task of Bitcoin Miners</span>
Bitcoin depends on miners to
- store and broadcast the block chain
- validate new transactions
- vote (by hash power) on consensus

Mining Bitcoin:
1. Join the network, listen for transactions, and validate all proposed transactions.
2. Listen for new blocks, maintain block chain.
3. Assemble a new valid block.
4. Find the nonce to make the block valid.
5. Hope everybody accepts your block.
6. Get block profit.

Set the mining difficulty:

Every two weeks, compute:
$$next_difficulty=previous_difficulty*(2 weeks)/(time to mine last 2016 blocks)$$
Expected time: 10 minutes / block
##### <span style="color:green">Mining pools</span>
Goal: Pool participants all attempt to mine a block with the same coinbase recipient. The recipient the ndistributes revenues to members based on work performed.

Mining pool variations:

Pay per share: flat reward per share
- Typically minus a significant fee
- Miners may never send in valid blocks.

Proportional: typically since last block
- lower risk for manager
- more work to verify

Luke-jr: no management fee
- Miners can only get paid out in whole Bitcoin.
- Pool owner keeps spread.

Pros:
- Make mining more predictable.
- Allow small miners to participate.
- More miners using updated validation software.

Cons:
- Lead to centralization.
- Discourage miners from running full nodes.

##### <span style="color:green">Mining incentives and strategies</span>
Strategic decisions:
- Which transactions to include in a block? Default: any above minumum transaction fee
- Which block to mine on top of? Default: longest valid chain
- How to choose between colliding blocks? Default: first block heard
- When to announce new blocks? Default: immediately after finding them

Forking attacks are possible if someone controls majority of the computing power.

The forking attacks are detectable. The community can reverse them by rejecting the blocks. The attacks might crash the exchange rate.

Checkpointing to avoid forking attacks (involve centralization)

Block-withholding attacks: not announce blocks right away (selfish mining)

Punitive forking: Block transactions from certain addresses.

With computing power control < 0.5, the attack falls behind the network soon.

Feather-forking: Announce to refuse mine directly on any blocks with a transaction from certain addresses, but concede after n confirming blocks.

Chance of pruning an offending block is squared (cause other miners to join.)

Feather forking is good for
- freezing individual bitcoin owners (ransom / extorsion, law enforcement)
- enforcing a minimum transaction fee

As a matter of transaction fee,

For default policy: $priority = sum(input\_value*input\_age)/size\_in\_bytes$, accept if $priority > 0.576$

Currently, block rewards are > 99%, but eventually transaction fees will dominate.

### <span style="color:red">Week 6</span>
##### <span style="color:green">Bitcoin Anonymity</span>
Bitcoin addresses are public key hashes (pseudonymity) rather than real identities.

anonymity = pseudonymity + unlinkability

Unlinkability: Different interactions of the same user with the system should not be linkable to each other.

Why is unlinkability needed in Bitcoin?
- Many Bitcoin services require real identity.
- Linked profiles can be deanonymized by a variety of side channels.

Unlinkability requirement of Bitcoin:
- Hard to link different addresses of the same user.
- Hard to link different transactions of the same user.
- Hard to link sender of a payment to its recipient.

Calculate anonymity set to quantify anonymity.

Why cryptocurrencies need anonymity?

Blockchain based currencies are totally, publicly, and permanently traceable.

Anonymity & decentralization are in conflict.
- Interactive protocols with bank are hard to decentralize.
- Dencentralization often achieved via public tracebility to enforce security.

##### <span style="color:green">De-anonymize Bitcoin</span>
Shared spending is evidence of joint control. So addresses can be linked transitively.

Idiosyncratic features of wallet software.

Each address used only once as change.

The first node to inform you of a transaction is probably the source of it.

Network-layer de-anonymization: Tor

Tor is intended for low-latency activities. (Mix nets might provide better anonymity, but Tor is what's deployed and works.)
##### <span style="color:green">Mixing</span>
Mixing: Use an intermediary to protect anonymity.

Dedicated mixing services promise not to keep records, and not to ask for your identity. Meanwhile online wallets typically require identity, keep records. Users trust them with their bitcoins and keep them for longer bigger anonymity set w.r.t. everyone else.

Principles for mixing services:
1. Use a series of mixes. Mixes should implement a standard API to make this easy.
2. Uniform transactions. In particular: all mix transactions must have the same value.
3. Client side must be automated and built-in to desktop wallet software.
4. Fees must be all-or-nothing. It can be implemented as probabilistic fees (with probability to swallow all the chunk.)

##### <span style="color:green">Decentralized Mixing</span>
Motives of decentralized mixing:
- No bootstrapping problem
- Theft impossible
- Possibly better anonymity
- More philosophically aligned with Bitcoin

Coinjoin provides a single transaction that has inputs from different users and different outputs corresponding to the users, but has random order.

Coinjoin algorithm:
1. Find peers who want to mix.
2. Exchange input / output addresses.
3. Construct transaction.
4. Send it around, collect signatures (Before signing, each peer checks if the output is present.)
5. Broadcast the transaction.

Strawman solution of peer anonymity:
1. exchange inputs
2. disconnect and reconnect over Tor
3. exchange outputs

Better solution: special-purpose anonymous routing mechanism

Merge avoidance: Instead of a single payment transaction, receiver provides multiple output addresses, and sender avoids combining different inputs.
##### <span style="color:green">Zerocoin and Zerocash</span>
Zerocoin provides protocol-level mixing: Mxing capability baked into protocol.
- Advatnage: cryptographic gurantee of mixing.
- Disadvantage: not currently compatible with Bitcoin.

Basecoin is a Bitcoin-like Altcoin. Zerocoin is an extension of Basecoin. Basecoins can be converted into Zerocoins and back by breaking link between original and new basecoins.

A Zerocoin is a cryptographic proof that you owned a basecoin and made it unspendable. Miners can verify the proofs. It gives the right to redeem a new Basecoin.

Minting a zerocoin: "commitment"
1. Generate serial number $S$ (eventually public)
2. Generate random secret $r$ (never public)
3. Compute $H(S,r)$
4. Create Mint Tx with 1 basecoin as input.

To spend a zerocoin:
1. Reveal $S$ (miners verify it hasn't been spent before)
2. Create zero-knowledge proof: "I know a number $r$ such that $H(S,r)$ is one of the zerocoins in the blockchain."
3. Pick arbitrary zerocoin in blockchain and use as input to the new transaction.

Zerocoin is anonymous. Since $r$ is secret, no one can figure out which zerocoin corresponds to serial number $S$.

Zerocoin is efficient. The proof is a giant disjunction over all zerocoins, yet the proof is relatively small.

Zerocash: Zerocoin without Basecoin
- different crypto for proofs (more efficient)
- proposal to run system without Basecoin

In Zerocash, all transactions are zerocoins. Splitting and merging are supported by putting transaction value inside the envelope. Ledger merely records existence of transactions.

Random, secret inputs are required to generate public parameters. These secret inputs must then be securely destroyed. No one can know them (anyone does can break the system.)
##### <span style="color:green">Tor and the Silk Road</span>
A Tor client picks a random path to destination server.

System | Type | Anonymity | Deployability
- | - | - | -
Bitcoin | Pseudonymous | Tx graph analysis | Default
Single mix | Mix | Tx graph analysis, bad mix | Usable today
Mix Chain | Mix | Side channels, bad mixes / peers | Bitcoin-compatible
Zerocoin | Cryptographic mix | Side channels (possibly) | Altcoin
Zerocash | Untraceable | None | Altcoin, tricky setup

### <span style="color:red">Week 7</span>
##### <span style="color:green">Consensus in Bitcoin</span>
Bitcoin relies on three concensus.

Consensus about rules agree on:
- what makes a transaction valid.
- what makes a block valid.
- how P2P nodes should behave.
- protocols and formats.

Consensus about history agrees on contents of the blockchain and therefore which transactions have occurred and which coins exist and who owns them.

Consensus that coins are valuable (any currency needs it)
##### <span style="color:green">Stakeholders</span>
Bitcoin Core developers have the power. Almost everybody follows their rules.

Miners have the power. History will be consistent with miners' consensus rules.

inverstors have the power. Investors determine whether Bitcoin has any value. In case of hard-fork, inverstors decide which branch wins.

Mercahnts and customers have the power. They drive the long-term price of Bitcoin.

Payment services have the power. They are really handling transactions.

The Bitcoin Foundation pays the core developers talks to government as the "voice of Bitcoin."
##### <span style="color:green">Roots of Bitcoin</span>